In [ ]:
# Coupled Pendulums Simulation with Advanced Synchronization Diagnostics
# --------------------------------------------------------------------------------
# Two pendulums of length L and mass m hang from separate pivots separated by distance d,
# connected by a spring of constant k and rest length D, with damping b.
# This notebook cell animates the system and provides multiple diagnostics to detect
# the onset of synchronization robustly:
#  1) Angle time series (θ₁, θ₂)\#  2) Sliding-window correlation
#  3) Absolute phase difference |θ₁ - θ₂| (wrapped)
#  4) Kuramoto order parameter r(t)
#  5) Instantaneous frequency difference Δω = ω₁ - ω₂
#  6) Amplitude envelope ratio via Hilbert transform
#  7) Phase portrait with time color gradient
#  8) Kinetic energy ratio E₁/E₂

import numpy as np
from scipy.integrate import solve_ivp
from scipy.signal import hilbert
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Circle
from IPython.display import HTML, display

# === USER-ADJUSTABLE PARAMETERS ===
# Physical parameters
m = 1.0       # mass (kg)
L = 1.0       # pendulum length (m)
d = 1.0       # pivot separation (m)
k = 5.0       # spring constant (N/m)
D = 0.8       # spring rest length (m)
b = 1e-3    # damping coeff (kg·m²/s)
g = 9.81      # gravity (m/s²)

# Initial state [θ₁, ω₁, θ₂, ω₂]
y0 = [1.0, 0.0, 0.7, 0.0]  # e.g. 57° vs 40° for gradual sync

# Simulation timing
t_span = (0, 60)      # seconds
num_steps = 3000      # resolution
# === end parameters ===

o1 = np.array([-d/2, 0.0])
o2 = np.array([ d/2, 0.0])
t = np.linspace(t_span[0], t_span[1], num_steps)

# Equations of motion
def deriv(t, y):
    θ1, ω1, θ2, ω2 = y
    p1 = o1 + [L*np.sin(θ1), -L*np.cos(θ1)]
    p2 = o2 + [L*np.sin(θ2), -L*np.cos(θ2)]
    dp = p2 - p1
    dist = np.hypot(*dp)
    Fs = k*(dist - D)
    u = dp/dist
    # tangential unit vectors
    t1 = [np.cos(θ1), np.sin(θ1)]
    t2 = [np.cos(θ2), np.sin(θ2)]
    Ft1 = Fs*np.dot(u, t1)
    Ft2 = -Fs*np.dot(u, t2)
    α1 = -(g/L)*np.sin(θ1) + Ft1/(m*L) - (b/(m*L**2))*ω1
    α2 = -(g/L)*np.sin(θ2) + Ft2/(m*L) - (b/(m*L**2))*ω2
    return [ω1, α1, ω2, α2]

# Integrate system
t_eval = t
sol = solve_ivp(deriv, t_span, y0, t_eval=t_eval)
θ1, ω1 = sol.y[0], sol.y[1]
θ2, ω2 = sol.y[2], sol.y[3]

# Compute positions for animation
p1 = o1 + np.vstack([L*np.sin(θ1), -L*np.cos(θ1)]).T
p2 = o2 + np.vstack([L*np.sin(θ2), -L*np.cos(θ2)]).T

# --- Animation ---
fig, ax = plt.subplots(figsize=(7,6))
margin = 1.2*L + d/2
ax.set_xlim(-margin, margin); ax.set_ylim(-margin, margin/1.2)
ax.set_aspect('equal'); ax.set_axisbelow(True)
ax.grid('--',linewidth=0.5); ax.set_xticks([]); ax.set_yticks([])
ax.plot(*o1,'ko'); ax.plot(*o2,'ko')
rod1,=ax.plot([],[],lw=3); rod2,=ax.plot([],[],lw=3)
b1=Circle((0,0),0.08,fc='blue'); b2=Circle((0,0),0.08,fc='red')
ax.add_patch(b1); ax.add_patch(b2)
s_line,=ax.plot([],[],lw=2,linestyle='--')

def init_anim():
    rod1.set_data([],[]); rod2.set_data([],[])
    s_line.set_data([],[])
    b1.center=(0,0); b2.center=(0,0)
    return rod1,rod2,s_line,b1,b2

def update_anim(i):
    P1, P2 = p1[i], p2[i]
    rod1.set_data([o1[0],P1[0]],[o1[1],P1[1]]);
    rod2.set_data([o2[0],P2[0]],[o2[1],P2[1]]);
    b1.center=P1; b2.center=P2
    s_line.set_data([P1[0],P2[0]],[P1[1],P2[1]])
    return rod1,rod2,s_line,b1,b2

anim = FuncAnimation(fig, update_anim, frames=len(t), init_func=init_anim, interval=20, blit=True)
display(HTML(anim.to_jshtml()))
plt.close(fig)

# --- Diagnostics: Advanced Metrics: Smoothed and Focused ---
# Time array
tp = t

# 1) Sliding-window correlation
def sliding_corr(x, y, frac=0.1):
    N = len(x)
    win = int(frac * N)
    c = np.full(N, np.nan)
    for i in range(win, N):
        c[i] = np.corrcoef(x[i-win:i], y[i-win:i])[0,1]
    return c
corr = sliding_corr(θ1, θ2)

# 2) Absolute phase difference (wrapped)
phase_diff = np.mod(np.unwrap(θ1) - np.unwrap(θ2) + np.pi, 2*np.pi) - np.pi

# 3) Kuramoto order parameter r(t)
order_param = np.abs((np.exp(1j*θ1) + np.exp(1j*θ2)) / 2)

# 4) Instantaneous frequency difference Δω
freq_diff = ω1 - ω2

# 5) Amplitude envelope via Hilbert transform
analytic1 = hilbert(θ1)
analytic2 = hilbert(θ2)
amp1 = np.abs(analytic1)
amp2 = np.abs(analytic2)
# Ratio and mismatch metrics
amp_ratio = amp1 / (amp2 + 1e-8)
amp_mismatch = np.abs(amp_ratio - 1)

# 6) Kinetic energy ratio and smoothing
E1 = 0.5 * m * (L * ω1)**2
E2 = 0.5 * m * (L * ω2)**2
ek_ratio = np.where(E2 > 1e-8, E1 / E2, np.nan)
# Smooth with moving average to reduce outliers
def smooth(x, window_len=50):
    kernel = np.ones(window_len) / window_len
    return np.convolve(x, kernel, mode='same')
smooth_ek = smooth(ek_ratio)
# Clip extreme values for visualization
smooth_ek = np.clip(smooth_ek, 0, 2)

# Diagnostics dictionary updated
diagnostics = {
    'Sliding Correlation': (tp, corr, None),
    'Phase Difference':   (tp, phase_diff, None),
    'Order Parameter r(t)': (tp, order_param, None),
    'Freq Diff Δω':       (tp, freq_diff, None),
    'Amp Mismatch |A₁/A₂ - 1|': (tp, amp_mismatch, None),
    'Energy Ratio (smoothed)': (tp, smooth_ek, None)
}

# Plot key diagnostics in a 3x2 grid
fig2, axs = plt.subplots(3,2, figsize=(12,10))
axs = axs.flatten()
for ax, (title, (x, y, lbls)) in zip(axs, diagnostics.items()):
    ax.plot(x, y)
    ax.set_title(title)
    ax.grid(True)
    # Mark synchronization threshold lines
    if title == 'Order Parameter r(t)':
        ax.axhline(1.0, color='gray', linestyle='--', linewidth=1)
    if title == 'Amp Mismatch |A₁/A₂ - 1|':
        ax.axhline(0.0, color='gray', linestyle='--', linewidth=1)
plt.tight_layout()


Animation size has reached 20974166 bytes, exceeding the limit of 20971520.0. If you're sure you want a larger animation embedded, set the animation.embed_limit rc parameter to a larger value (in MB). This and further frames will be dropped.
